In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore')

args, namespace ['inline'] None
Namespace(gui='inline', list=False) []


In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
# 检查缺失值数据
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
# 删除缺失值所在行
df.dropna(inplace=True)

In [6]:
df['StockCode']= df['StockCode'].astype(str)

In [7]:
# 让我们来看看我们的数据集中消费者的数量:

customers = df["CustomerID"].unique().tolist()
len(customers)

4372

In [8]:
# 在我们的数据集中有4,372个消费者,对于这些消费者，我们将提取他们的购买历史。换句话说，我们可以有4372个购买序列。

# 留出数据集的一小部分用于验证是一个很好的方法。因此，我将使用90%消费者的数据来创建word2vec embeddings。让我们开始分割数据。

In [9]:
# 打乱消费者id
random.shuffle(customers)
# 提取90%的消费者
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]
# 分为训练集和验证集
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

In [10]:
# 存储消费者的购买历史
purchases_train = []
# 用商品代码填充列表(应该是按照时间先后顺序的才对？？)
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)

100%|█████████████████████████████████████████████████████████████████████████████| 3935/3935 [00:04<00:00, 856.92it/s]


In [12]:
# purchases_train[:3]

In [13]:
# 存储消费者的购买历史
purchases_val = []
# 用商品代码填充列表
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 437/437 [00:00<00:00, 1132.38it/s]


In [14]:
# 为商品构建word2vec Embeddings
# 训练word2vec模型
model = Word2Vec(window = 10, sg = 1, hs = 0,
negative = 10, # for negative sampling
alpha=0.03, min_alpha=0.0007,
seed = 14)
model.build_vocab(purchases_train, progress_per=200)
model.train(purchases_train, total_examples = model.corpus_count, 
epochs=10, report_delay=1)

(3627787, 3664060)

In [15]:
# 让我们来看看“model”的相关参数:
print(model)

Word2Vec(vocab=3179, size=100, alpha=0.03)


In [16]:
# 我们的模型有3179个唯一的单词，每个单词的向量大小为100维。接下来，我们将提取词汇表中所有单词的向量，并将其存储在一个地方，以便于访问。

In [17]:
# 提取向量
X = model[model.wv.vocab]
X.shape

(3179, 100)

In [18]:
# 可视化word2vec Embeddings
# 可视化你所创建的embeddings是很有帮助的。在这里，我们有100维的Embeddings。我们甚至无法可视化4维空间，更不用说100维了，那么我们怎么做呢?

# 我们将使用UMAP算法将商品Embeddings的维数从100降到2，UMAP算法通常用于降维

In [21]:
import umap
cluster_embedding = umap.UMAP(n_neighbors=30, min_dist=0.0,
n_components=2, random_state=42).fit_transform(X)
plt.figure(figsize=(10,9))
plt.scatter(cluster_embedding[:, 0], cluster_embedding[:, 1], s=3, cmap='Spectral')


AttributeError: module 'umap' has no attribute 'UMAP'

In [22]:
# 让我们首先创建一个商品id和商品描述的字典，以便轻松地将商品的描述映射到其id，反之亦然。

products = train_df[["StockCode", "Description"]]
# 去重
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")
# 创建一个商品id和商品描述的字典
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [23]:
# 字典测试
products_dict['84029E']

['RED WOOLLY HOTTIE WHITE HEART.']

In [25]:
# 我定义了下面的函数。将一个商品的向量(n)作为输入，返回前6个相似的商品:

def similar_products(v, n = 6):
# 为输入向量提取最相似的商品
    ms = model.similar_by_vector(v, topn= n+1)[1:]
# 提取相似产品的名称和相似度评分
    new_ms = []
    for j in ms:
            pair = (products_dict[j[0]][0], j[1])
            new_ms.append(pair)
    return new_ms 

In [26]:
# 让我们通过传递商品编号为'90019A' (‘SILVER M.O.P ORBIT BRACELET’)的商品:

similar_products(model['90019A'])

[('SILVER M.O.P ORBIT DROP EARRINGS', 0.7453106641769409),
 ('GOLD/M.O.P PENDANT ORBIT NECKLACE', 0.7362282276153564),
 ('PINK BOUDICCA LARGE BRACELET', 0.7348238825798035),
 ('AMBER DROP EARRINGS W LONG BEADS', 0.7338213920593262),
 ('PINK HEART OF GLASS BRACELET', 0.7186692357063293),
 ('SILVER LARIAT BLACK STONE EARRINGS', 0.7179129123687744)]

In [27]:
# 银色M.O.P轨道手链
# 银色M.O.P轨道吊坠耳环
# G / M.O.P吊坠项链
# 粉色BOUDICCA大号手链
# 琥珀吊坠耳环，长珠子
# 粉色心形玻璃手链
# 银LARIAT黑石耳环

In [28]:
# 太酷了!结果还是非常相关，并且与输入商品匹配得很好。然而，这个输出仅基于单个商品的向量。如果我们想根据他或她过去的多次购买来推荐商品呢?

# 一个简单的解决方案是取用户迄今为止购买的所有商品的向量的平均值，并使用这个结果向量找到类似的商品。我们将使用下面的函数，它接收一个商品id列表，并返回一个100维的向量，它是输入列表中商品的向量的平均值:

In [31]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
    return np.mean(product_vec, axis=0)

In [32]:
# 回想一下，为了验证目的，我们已经创建了一个单独的购买序列列表。现在刚好可以利用它。

In [33]:
# 用户购买的第一个商品列表的长度为314。我们将把这个验证集的商品序列传递给aggregate_vectors函数。

aggregate_vectors(purchases_val[0]).shape

(100,)

In [34]:
# 函数返回了一个100维的数组。这意味着函数运行正常。现在我们可以用这个结果得到最相似的商品:

similar_products(aggregate_vectors(purchases_val[0]))

[('RED RETROSPOT PICNIC BAG', 0.6597121953964233),
 ('JUMBO BAG ALPHABET', 0.6510666608810425),
 ('SPOTTY BUNTING', 0.6495421528816223),
 ('JUMBO STORAGE BAG SUKI', 0.6465193629264832),
 ('JUMBO BAG VINTAGE DOILY ', 0.6461219191551208),
 ('LUNCH BAG VINTAGE DOILY ', 0.6446658968925476)]

In [35]:
# RED RETROSPOT PICNIC BAG
# JUMBO BAG ALPHABET
# SPOTTY BUNTING
# JUMBO STORAGE BAG SUKI
# JUMBO BAG VINTAGE DOILY 
# LUNCH BAG VINTAGE DOILY 

# 红色TROTROPOT野餐包
# 巨型手提袋
# 弹跳
# 巨型存储袋SUKI
# 巨型手提袋古董
# 午餐袋复古风格

In [36]:
# 结果，我们的系统根据用户的整个购买历史推荐了6款商品。此外，你也可以根据最近几次购买情况来进行商品推荐。

# 下面我只提供了最近购买的10种商品作为输入:

In [40]:
similar_products(aggregate_vectors(purchases_val[0][-10:]))

[('JUMBO BAG VINTAGE DOILY ', 0.7394315004348755),
 ("JUMBO BAG 50'S CHRISTMAS ", 0.7153654098510742),
 ('JUMBO BAG PAISLEY PARK', 0.7148979902267456),
 ('JUMBO BAG PEARS', 0.7023414969444275),
 ('JUMBO BAG VINTAGE LEAF', 0.6900845766067505),
 ('JUMBO BAG ALPHABET', 0.6774144768714905)]

In [38]:
# 你可以随意修改这段代码，并尝试从验证集中的更多商品序列进行商品推荐。也可以进一步优化这段代码或使其更好。